In [118]:
import json
import pandas as pd
from datetime import datetime
from listas import priorfalls_list, delirium_list, dementia_list, depresyndr_list, uriincont_list, fecincont_list, pressulc_list, osteopor_list, sarcopenia_list, sleepdisturb_list, chrpain_list, iatrog_list, constipation_list, CVdisease_list, heartdisease_list, ND_list, DM_list, hepatopat_list, neopl_list, AcuteRenalF_list, dizsyn_list, VIH_list, psicosis_list

from auxiliary_functions import valores_codigos, contar_diccionarios, dias_ingreso_total, asignar_intervalo_edad, sumar_barthel, sumar_emina, obtener_ultimo_resultat, obtener_valor_promedio, canadenca_comparada, disfagia_mecvvs, extraer_valor_clave, extraer_valor_clave_simple

tabla = pd.read_pickle('./data/processed/dataframe.pkl')
import scipy.stats as stats
from scipy.stats import ttest_ind
from scipy.stats import shapiro
from scipy.stats import mannwhitneyu
from scipy.stats import chi2_contingency


In [119]:
tabla.head()

,edat,sexe,visitaDietista,labs,mna,emina,canadenca,pes,atcs,mecvvs,...,albumina,proteinas totales,Hb,colesterol total,leucos,limfos,prot C react,urea,FGE MDRD,FGE CDK-EPI
0,89,F,0,[],[],"[{'activitat': '2', 'estatMental': '1', 'humit...",[],[],"[{'codi': 'M01AE', 'dataInici': '2020-06-21 00...",[],...,None,None,None,None,None,None,None,None,None,None
1,70,M,0,"[{'date': '20220524', 'name': 'PCR SARS-CoV2 S...",[],"[{'activitat': '0', 'estatMental': '0', 'humit...",[],"[{'valor': '64.7', 'data': '2022-10-04'}, {'va...","[{'codi': 'C09AA', 'dataInici': '2021-05-26 00...","[{'data': '20220930', 'pacientNoColabora': '',...",...,3.1,5.3,14.5,124.7,X,38.9,19.61,25.7,>60,>90
2,96,M,0,[],[],"[{'activitat': '3', 'estatMental': '0', 'humit...",[],[],"[{'codi': 'G04CA', 'dataInici': '2020-07-09 00...",[],...,None,None,None,None,None,None,None,None,None,None
3,85,M,0,[],[],"[{'activitat': '2', 'estatMental': '1', 'humit...",[],[],"[{'codi': 'G04CA', 'dataInici': '2020-07-15 00...","[{'data': '20201204131810', 'disfagia': 'S', '...",...,None,None,None,None,None,None,None,None,None,None
4,93,M,0,"[{'date': '20220208', 'name': 'UREA Sèrum', 'l...","[{'imc': '0', 'anorexia': '1', 'perduaPes': '1...","[{'activitat': '2', 'estatMental': '1', 'humit...",[],"[{'valor': '49.900', 'data': '2022-06-02'}, {'...","[{'codi': 'N02AA', 'dataInici': '2022-02-01 00...","[{'data': '20220330172303', 'disfagiaConeguda'...",...,Proves afegides a un altre petició per extracc...,Proves afegides a un altre petició per extracc...,X,None,X,X,0.54,Proves afegides a un altre petició per extracc...,Proves afegides a un altre petició per extracc...,Proves afegides a un altre petició per extracc...


In [120]:
CON_PA = tabla[tabla["PA_diagnosticada"] == 1.0]
CON_PA_MECVV = tabla[(tabla["P_diagnosticada"] == 1.0) & (tabla["Disfagia_mecvvs"] == 1.0) & (tabla["Dias_totales_ingresado"] < 30)]
SIN_PA = tabla[(tabla["P_diagnosticada"] == 1.0) & (tabla["Disfagia_mecvvs"] == 1.0) & (tabla["Dias_totales_ingresado"] > 30) | (tabla["P_diagnosticada"] == 1.0)]

# Sindromes geriatricos

## Indice de Charslton

## Caidas previas

In [121]:
caprev_CON_PA= (CON_PA['caidas_previas'] == 1.0).mean() * 100
print(f"Porcentaje de caidas previas: {caprev_CON_PA:.2f}%")

Porcentaje de caidas previas: 5.88%


In [122]:
caprev_CON_PA_MECVV= (CON_PA_MECVV['caidas_previas'] == 1.0).mean() * 100
print(f"Porcentaje de caidas previas: {caprev_CON_PA_MECVV:.2f}%")

Porcentaje de caidas previas: 0.00%


In [123]:
caprev_SIN_PA= (SIN_PA['caidas_previas'] == 1.0).mean() * 100
print(f"Porcentaje de caidas previas: {caprev_SIN_PA:.2f}%")

Porcentaje de caidas previas: 6.12%


#### Pvalor

##### Normalidad

In [124]:
stat, p_value = shapiro(CON_PA['caidas_previas'])
print(f"Estadístico W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El número de filas no sigue una distribución normal (rechazar H0).")
else:
    print("El número de filas sigue una distribución normal (aceptar H0).")

Estadístico W = 0.2622
Valor p = 0.0000
El número de filas no sigue una distribución normal (rechazar H0).


In [125]:
stat, p_value = shapiro(CON_PA_MECVV['caidas_previas'])
print(f"Estadístico W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El número de filas no sigue una distribución normal (rechazar H0).")
else:
    print("El número de filas sigue una distribución normal (aceptar H0).")

Estadístico W = 1.0000
Valor p = 1.0000
El número de filas sigue una distribución normal (aceptar H0).


C:\Users\maria\PycharmProjects\practicum\.venv\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: Input data has range zero. The results may not be accurate.
  res = hypotest_fun_out(*samples, **kwds)


In [126]:
stat, p_value = shapiro(SIN_PA['DO_diagnosticada'])
print(f"Estadístico W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El número de filas no sigue una distribución normal (rechazar H0).")
else:
    print("El número de filas sigue una distribución normal (aceptar H0).")

Estadístico W = 0.4716
Valor p = 0.0000
El número de filas no sigue una distribución normal (rechazar H0).


In [127]:
stat, p_value_12 = mannwhitneyu(CON_PA['caidas_previas'], CON_PA_MECVV['caidas_previas'], alternative='two-sided')
print("P-valor entre CON_PA y CON_PA_MECVV", p_value_12)

P-valor entre CON_PA y CON_PA_MECVV 0.49002535950179893


In [128]:
stat, p_value_13 = mannwhitneyu(CON_PA['caidas_previas'], SIN_PA['caidas_previas'], alternative='two-sided')
print("P-valor entre CON_PA y SIN_PA", p_value_13)

P-valor entre CON_PA y SIN_PA 0.9858477456716862


In [129]:
stat, p_value_23 = mannwhitneyu(CON_PA_MECVV['caidas_previas'], SIN_PA['caidas_previas'], alternative='two-sided')
print("P-valor entre CON_PA_MECVV y SIN_PA", p_value_23)

P-valor entre CON_PA_MECVV y SIN_PA 0.44143809205435813


## Delirios

In [130]:
del_CON_PA= (CON_PA['delirios'] == 1.0).mean() * 100
print(f"Porcentaje de delirios: {del_CON_PA:.2f}%")

Porcentaje de delirios: 23.53%


In [131]:
del_CON_PA_MECVV= (CON_PA_MECVV['delirios'] == 1.0).mean() * 100
print(f"Porcentaje de delirios: {del_CON_PA_MECVV:.2f}%")

Porcentaje de delirios: 40.00%


In [132]:
del_SIN_PA= (SIN_PA['delirios'] == 1.0).mean() * 100
print(f"Porcentaje de delirios: {del_SIN_PA:.2f}%")

Porcentaje de delirios: 18.37%


#### Pvalor

##### Normalidad

In [133]:
stat, p_value = shapiro(CON_PA['delirios'])
print(f"Estadístico W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El número de filas no sigue una distribución normal (rechazar H0).")
else:
    print("El número de filas sigue una distribución normal (aceptar H0).")

Estadístico W = 0.5327
Valor p = 0.0000
El número de filas no sigue una distribución normal (rechazar H0).


In [134]:
stat, p_value = shapiro(CON_PA_MECVV['delirios'])
print(f"Estadístico W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El número de filas no sigue una distribución normal (rechazar H0).")
else:
    print("El número de filas sigue una distribución normal (aceptar H0).")

Estadístico W = 0.6405
Valor p = 0.0002
El número de filas no sigue una distribución normal (rechazar H0).


In [135]:
stat, p_value = shapiro(SIN_PA['delirios'])
print(f"Estadístico W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El número de filas no sigue una distribución normal (rechazar H0).")
else:
    print("El número de filas sigue una distribución normal (aceptar H0).")

Estadístico W = 0.4716
Valor p = 0.0000
El número de filas no sigue una distribución normal (rechazar H0).


In [136]:
stat, p_value_12 = mannwhitneyu(CON_PA['delirios'], CON_PA_MECVV['delirios'], alternative='two-sided')
print("P-valor entre CON_PA y CON_PA_MECVV", p_value_12)

P-valor entre CON_PA y CON_PA_MECVV 0.39174655228054167


In [137]:
stat, p_value_13 = mannwhitneyu(CON_PA['delirios'], SIN_PA['delirios'], alternative='two-sided')
print("P-valor entre CON_PA y SIN_PA", p_value_13)

P-valor entre CON_PA y SIN_PA 0.6549000779898075


In [138]:
stat, p_value_23 = mannwhitneyu(CON_PA_MECVV['delirios'], SIN_PA['delirios'], alternative='two-sided')
print("P-valor entre CON_PA_MECVV y SIN_PA", p_value_23)

P-valor entre CON_PA_MECVV y SIN_PA 0.13960802912048415


## Demencia

In [139]:
dem_CON_PA= (CON_PA['demencia'] == 1.0).mean() * 100
print(f"Porcentaje de demencia: {dem_CON_PA:.2f}%")

Porcentaje de demencia: 47.06%


In [140]:
dem_CON_PA_MECVV= (CON_PA_MECVV['demencia'] == 1.0).mean() * 100
print(f"Porcentaje de demencia: {dem_CON_PA_MECVV:.2f}%")

Porcentaje de demencia: 40.00%


In [141]:
dem_SIN_PA= (SIN_PA['demencia'] == 1.0).mean() * 100
print(f"Porcentaje de demencia: {dem_SIN_PA:.2f}%")

Porcentaje de demencia: 30.61%


#### Pvalor

##### Normalidad

In [142]:
stat, p_value = shapiro(CON_PA['demencia'])
print(f"Estadístico W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El número de filas no sigue una distribución normal (rechazar H0).")
else:
    print("El número de filas sigue una distribución normal (aceptar H0).")

Estadístico W = 0.6418
Valor p = 0.0000
El número de filas no sigue una distribución normal (rechazar H0).


In [143]:
stat, p_value = shapiro(CON_PA_MECVV['demencia'])
print(f"Estadístico W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El número de filas no sigue una distribución normal (rechazar H0).")
else:
    print("El número de filas sigue una distribución normal (aceptar H0).")

Estadístico W = 0.6405
Valor p = 0.0002
El número de filas no sigue una distribución normal (rechazar H0).


In [144]:
stat, p_value = shapiro(SIN_PA['delirios'])
print(f"Estadístico W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El número de filas no sigue una distribución normal (rechazar H0).")
else:
    print("El número de filas sigue una distribución normal (aceptar H0).")

Estadístico W = 0.4716
Valor p = 0.0000
El número de filas no sigue una distribución normal (rechazar H0).


In [145]:
stat, p_value_12 = mannwhitneyu(CON_PA['demencia'], CON_PA_MECVV['demencia'], alternative='two-sided')
print("P-valor entre CON_PA y CON_PA_MECVV", p_value_12)

P-valor entre CON_PA y CON_PA_MECVV 0.7484826918143946


In [146]:
stat, p_value_13 = mannwhitneyu(CON_PA['demencia'], SIN_PA['demencia'], alternative='two-sided')
print("P-valor entre CON_PA y SIN_PA", p_value_13)

P-valor entre CON_PA y SIN_PA 0.22703869678020527


In [147]:
stat, p_value_23 = mannwhitneyu(CON_PA_MECVV['demencia'], SIN_PA['demencia'], alternative='two-sided')
print("P-valor entre CON_PA_MECVV y SIN_PA", p_value_23)

P-valor entre CON_PA_MECVV y SIN_PA 0.5743933608939569


## Sindrome depresivo 

In [148]:
depre_CON_PA= (CON_PA['sindrome_depresivo'] == 1.0).mean() * 100
print(f"Porcentaje de sindrome depresivo: {depre_CON_PA:.2f}%")

Porcentaje de sindrome depresivo: 23.53%


In [149]:
depre_CON_PA_MECVV= (CON_PA_MECVV['sindrome_depresivo'] == 1.0).mean() * 100
print(f"Porcentaje de sindrome depresivo: {depre_CON_PA_MECVV:.2f}%")

Porcentaje de sindrome depresivo: 0.00%


In [150]:
depre_SIN_PA= (SIN_PA['sindrome_depresivo'] == 1.0).mean() * 100
print(f"Porcentaje de sindrome depresivo: {depre_SIN_PA:.2f}%")

Porcentaje de sindrome depresivo: 14.29%


#### Pvalor

##### Normalidad

In [151]:
stat, p_value = shapiro(CON_PA['sindrome_depresivo'])
print(f"Estadístico W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El número de filas no sigue una distribución normal (rechazar H0).")
else:
    print("El número de filas sigue una distribución normal (aceptar H0).")

Estadístico W = 0.5327
Valor p = 0.0000
El número de filas no sigue una distribución normal (rechazar H0).


In [152]:
stat, p_value = shapiro(CON_PA_MECVV['sindrome_depresivo'])
print(f"Estadístico W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El número de filas no sigue una distribución normal (rechazar H0).")
else:
    print("El número de filas sigue una distribución normal (aceptar H0).")

Estadístico W = 1.0000
Valor p = 1.0000
El número de filas sigue una distribución normal (aceptar H0).


In [153]:
stat, p_value = shapiro(SIN_PA['sindrome_depresivo'])
print(f"Estadístico W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El número de filas no sigue una distribución normal (rechazar H0).")
else:
    print("El número de filas sigue una distribución normal (aceptar H0).")

Estadístico W = 0.4168
Valor p = 0.0000
El número de filas no sigue una distribución normal (rechazar H0).


In [154]:
stat, p_value_12 = mannwhitneyu(CON_PA['sindrome_depresivo'], CON_PA_MECVV['sindrome_depresivo'], alternative='two-sided')
print("P-valor entre CON_PA y CON_PA_MECVV", p_value_12)

P-valor entre CON_PA y CON_PA_MECVV 0.11180527770715322


In [155]:
stat, p_value_13 = mannwhitneyu(CON_PA['sindrome_depresivo'], SIN_PA['sindrome_depresivo'], alternative='two-sided')
print("P-valor entre CON_PA y SIN_PA", p_value_13)

P-valor entre CON_PA y SIN_PA 0.3880726918610017


In [156]:
stat, p_value_23 = mannwhitneyu(CON_PA_MECVV['sindrome_depresivo'], SIN_PA['sindrome_depresivo'], alternative='two-sided')
print("P-valor entre CON_PA_MECVV y SIN_PA", p_value_23)

P-valor entre CON_PA_MECVV y SIN_PA 0.21340049774345338


## Incontinencia urinaria

In [157]:
ic_CON_PA= (CON_PA['incont_uri'] == 1.0).mean() * 100
print(f"Porcentaje de incontinencia urinaria: {ic_CON_PA:.2f}%")

Porcentaje de incontinencia urinaria: 29.41%


In [158]:
ic_CON_PA_MECVV= (CON_PA_MECVV['incont_uri'] == 1.0).mean() * 100
print(f"Porcentaje de incontinencia urinaria: {ic_CON_PA_MECVV:.2f}%")

Porcentaje de incontinencia urinaria: 30.00%


In [159]:
ic_SIN_PA= (SIN_PA['incont_uri'] == 1.0).mean() * 100
print(f"Porcentaje de incontinencia urinaria: {ic_SIN_PA:.2f}%")

Porcentaje de incontinencia urinaria: 14.29%


#### Pvalor

##### Normalidad

In [160]:
stat, p_value = shapiro(CON_PA['incont_uri'])
print(f"Estadístico W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El número de filas no sigue una distribución normal (rechazar H0).")
else:
    print("El número de filas sigue una distribución normal (aceptar H0).")

Estadístico W = 0.5785
Valor p = 0.0000
El número de filas no sigue una distribución normal (rechazar H0).


In [161]:
stat, p_value = shapiro(CON_PA_MECVV['incont_uri'])
print(f"Estadístico W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El número de filas no sigue una distribución normal (rechazar H0).")
else:
    print("El número de filas sigue una distribución normal (aceptar H0).")

Estadístico W = 0.5942
Valor p = 0.0000
El número de filas no sigue una distribución normal (rechazar H0).


In [162]:
stat, p_value = shapiro(SIN_PA['incont_uri'])
print(f"Estadístico W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El número de filas no sigue una distribución normal (rechazar H0).")
else:
    print("El número de filas sigue una distribución normal (aceptar H0).")

Estadístico W = 0.4168
Valor p = 0.0000
El número de filas no sigue una distribución normal (rechazar H0).


In [163]:
stat, p_value_12 = mannwhitneyu(CON_PA['incont_uri'], CON_PA_MECVV['incont_uri'], alternative='two-sided')
print("P-valor entre CON_PA y CON_PA_MECVV", p_value_12)

P-valor entre CON_PA y CON_PA_MECVV 1.0


In [164]:
stat, p_value_13 = mannwhitneyu(CON_PA['incont_uri'], SIN_PA['incont_uri'], alternative='two-sided')
print("P-valor entre CON_PA y SIN_PA", p_value_13)

P-valor entre CON_PA y SIN_PA 0.17015925929193942


In [165]:
stat, p_value_23 = mannwhitneyu(CON_PA_MECVV['incont_uri'], SIN_PA['incont_uri'], alternative='two-sided')
print("P-valor entre CON_PA_MECVV y SIN_PA", p_value_23)

P-valor entre CON_PA_MECVV y SIN_PA 0.237514563226218


## Incontinencia fecal 

In [166]:
ifec_CON_PA= (CON_PA['incont_fec'] == 1.0).mean() * 100
print(f"Porcentaje de incontinencia fecal: {ifec_CON_PA:.2f}%")

Porcentaje de incontinencia fecal: 29.41%


In [167]:
ifec_CON_PA_MECVV= (CON_PA_MECVV['incont_fec'] == 1.0).mean() * 100
print(f"Porcentaje de incontinencia fecal: {ifec_CON_PA_MECVV:.2f}%")

Porcentaje de incontinencia fecal: 10.00%


In [168]:
ifec_SIN_PA= (SIN_PA['incont_fec'] == 1.0).mean() * 100
print(f"Porcentaje de incontinencia fecal: {ifec_SIN_PA:.2f}%")

Porcentaje de incontinencia fecal: 12.24%


#### Pvalor

##### Normalidad

In [169]:
stat, p_value = shapiro(CON_PA['incont_fec'])
print(f"Estadístico W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El número de filas no sigue una distribución normal (rechazar H0).")
else:
    print("El número de filas sigue una distribución normal (aceptar H0).")

Estadístico W = 0.5785
Valor p = 0.0000
El número de filas no sigue una distribución normal (rechazar H0).


In [170]:
stat, p_value = shapiro(CON_PA_MECVV['incont_fec'])
print(f"Estadístico W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El número de filas no sigue una distribución normal (rechazar H0).")
else:
    print("El número de filas sigue una distribución normal (aceptar H0).")

Estadístico W = 0.3657
Valor p = 0.0000
El número de filas no sigue una distribución normal (rechazar H0).


In [171]:
stat, p_value = shapiro(SIN_PA['incont_fec'])
print(f"Estadístico W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El número de filas no sigue una distribución normal (rechazar H0).")
else:
    print("El número de filas sigue una distribución normal (aceptar H0).")

Estadístico W = 0.3844
Valor p = 0.0000
El número de filas no sigue una distribución normal (rechazar H0).


In [172]:
stat, p_value_12 = mannwhitneyu(CON_PA['incont_fec'], CON_PA_MECVV['incont_fec'], alternative='two-sided')
print("P-valor entre CON_PA y CON_PA_MECVV", p_value_12)

P-valor entre CON_PA y CON_PA_MECVV 0.2649031747093591


In [173]:
stat, p_value_13 = mannwhitneyu(CON_PA['incont_fec'], SIN_PA['incont_fec'], alternative='two-sided')
print("P-valor entre CON_PA y SIN_PA", p_value_13)

P-valor entre CON_PA y SIN_PA 0.10681676423525503


In [174]:
stat, p_value_23 = mannwhitneyu(CON_PA_MECVV['incont_fec'], SIN_PA['incont_fec'], alternative='two-sided')
print("P-valor entre CON_PA_MECVV y SIN_PA", p_value_23)

P-valor entre CON_PA_MECVV y SIN_PA 0.8568943909879498


## Ulceras de presion

In [175]:
up_CON_PA= (CON_PA['ulceras_presion'] == 1.0).mean() * 100
print(f"Porcentaje de ulceras de presion: {up_CON_PA:.2f}%")

Porcentaje de ulceras de presion: 5.88%


In [176]:
up_CON_PA_MECVV= (CON_PA_MECVV['ulceras_presion'] == 1.0).mean() * 100
print(f"Porcentaje de ulceras de presion: {up_CON_PA_MECVV:.2f}%")

Porcentaje de ulceras de presion: 0.00%


In [177]:
up_SIN_PA= (SIN_PA['ulceras_presion'] == 1.0).mean() * 100
print(f"Porcentaje de ulceras de presion: {up_SIN_PA:.2f}%")

Porcentaje de ulceras de presion: 2.04%


#### Pvalor

##### Normalidad

In [178]:
stat, p_value = shapiro(CON_PA['ulceras_presion'])
print(f"Estadístico W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El número de filas no sigue una distribución normal (rechazar H0).")
else:
    print("El número de filas sigue una distribución normal (aceptar H0).")

Estadístico W = 0.2622
Valor p = 0.0000
El número de filas no sigue una distribución normal (rechazar H0).


In [179]:
stat, p_value = shapiro(CON_PA_MECVV['ulceras_presion'])
print(f"Estadístico W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El número de filas no sigue una distribución normal (rechazar H0).")
else:
    print("El número de filas sigue una distribución normal (aceptar H0).")

Estadístico W = 1.0000
Valor p = 1.0000
El número de filas sigue una distribución normal (aceptar H0).


In [180]:
stat, p_value = shapiro(SIN_PA['ulceras_presion'])
print(f"Estadístico W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El número de filas no sigue una distribución normal (rechazar H0).")
else:
    print("El número de filas sigue una distribución normal (aceptar H0).")

Estadístico W = 0.1273
Valor p = 0.0000
El número de filas no sigue una distribución normal (rechazar H0).


In [181]:
stat, p_value_12 = mannwhitneyu(CON_PA['ulceras_presion'], CON_PA_MECVV['ulceras_presion'], alternative='two-sided')
print("P-valor entre CON_PA y CON_PA_MECVV", p_value_12)

P-valor entre CON_PA y CON_PA_MECVV 0.49002535950179893


In [182]:
stat, p_value_13 = mannwhitneyu(CON_PA['ulceras_presion'], SIN_PA['ulceras_presion'], alternative='two-sided')
print("P-valor entre CON_PA y SIN_PA", p_value_13)

P-valor entre CON_PA y SIN_PA 0.4440308448923743


In [183]:
stat, p_value_23 = mannwhitneyu(CON_PA_MECVV['ulceras_presion'], SIN_PA['ulceras_presion'], alternative='two-sided')
print("P-valor entre CON_PA_MECVV y SIN_PA", p_value_23)

P-valor entre CON_PA_MECVV y SIN_PA 0.6843175494727258


## Immobilidad

## Confusion

## Osteoporosis

In [184]:
ost_CON_PA= (CON_PA['osteoporosis'] == 1.0).mean() * 100
print(f"Porcentaje de osteoporosis: {ost_CON_PA:.2f}%")

Porcentaje de osteoporosis: 0.00%


In [185]:
ost_CON_PA_MECVV= (CON_PA_MECVV['osteoporosis'] == 1.0).mean() * 100
print(f"Porcentaje de osteoporosis: {ost_CON_PA_MECVV:.2f}%")

Porcentaje de osteoporosis: 0.00%


In [186]:
ost_SIN_PA= (SIN_PA['osteoporosis'] == 1.0).mean() * 100
print(f"Porcentaje de osteoporosis: {ost_SIN_PA:.2f}%")

Porcentaje de osteoporosis: 0.00%


#### Pvalor

##### Normalidad

In [187]:
stat, p_value = shapiro(CON_PA['osteoporosis'])
print(f"Estadístico W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El número de filas no sigue una distribución normal (rechazar H0).")
else:
    print("El número de filas sigue una distribución normal (aceptar H0).")

Estadístico W = 1.0000
Valor p = 1.0000
El número de filas sigue una distribución normal (aceptar H0).


In [188]:
stat, p_value_12 = mannwhitneyu(CON_PA['osteoporosis'], CON_PA_MECVV['osteoporosis'], alternative='two-sided')
print("P-valor entre CON_PA y CON_PA_MECVV", p_value_12)

P-valor entre CON_PA y CON_PA_MECVV 1.0


In [189]:
stat, p_value_12 = ttest_ind(CON_PA['osteoporosis'], CON_PA_MECVV['osteoporosis'], equal_var= False)
print("P-valor entre CON_PA y CON_PA_MECVV", p_value_12)

P-valor entre CON_PA y CON_PA_MECVV nan


## Sarcopenia

In [190]:
sarc_CON_PA= (CON_PA['sarcopenia'] == 1.0).mean() * 100
print(f"Porcentaje de sarcopenia: {sarc_CON_PA:.2f}%")

Porcentaje de sarcopenia: 0.00%


In [191]:
sarc_CON_PA_MECVV= (CON_PA_MECVV['sarcopenia'] == 1.0).mean() * 100
print(f"Porcentaje de sarcopenia: {sarc_CON_PA_MECVV:.2f}%")

Porcentaje de sarcopenia: 10.00%


In [192]:
sarc_SIN_PA= (SIN_PA['sarcopenia'] == 1.0).mean() * 100
print(f"Porcentaje de sarcopenia: {sarc_SIN_PA:.2f}%")

Porcentaje de sarcopenia: 4.08%


#### Pvalor

##### Normalidad

In [193]:
stat, p_value = shapiro(CON_PA['sarcopenia'])
print(f"Estadístico W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El número de filas no sigue una distribución normal (rechazar H0).")
else:
    print("El número de filas sigue una distribución normal (aceptar H0).")

Estadístico W = 1.0000
Valor p = 1.0000
El número de filas sigue una distribución normal (aceptar H0).


In [194]:
stat, p_value = shapiro(CON_PA_MECVV['sarcopenia'])
print(f"Estadístico W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El número de filas no sigue una distribución normal (rechazar H0).")
else:
    print("El número de filas sigue una distribución normal (aceptar H0).")

Estadístico W = 0.3657
Valor p = 0.0000
El número de filas no sigue una distribución normal (rechazar H0).


In [195]:
stat, p_value = shapiro(SIN_PA['sarcopenia'])
print(f"Estadístico W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El número de filas no sigue una distribución normal (rechazar H0).")
else:
    print("El número de filas sigue una distribución normal (aceptar H0).")

Estadístico W = 0.2008
Valor p = 0.0000
El número de filas no sigue una distribución normal (rechazar H0).


In [196]:
stat, p_value_12 = mannwhitneyu(CON_PA['sarcopenia'], CON_PA_MECVV['sarcopenia'], alternative='two-sided')
print("P-valor entre CON_PA y CON_PA_MECVV", p_value_12)

P-valor entre CON_PA y CON_PA_MECVV 0.21976847950216105


In [197]:
stat, p_value_13 = mannwhitneyu(CON_PA['sarcopenia'], SIN_PA['sarcopenia'], alternative='two-sided')
print("P-valor entre CON_PA y SIN_PA", p_value_13)

P-valor entre CON_PA y SIN_PA 0.41519527301502746


In [198]:
stat, p_value_23 = mannwhitneyu(CON_PA_MECVV['sarcopenia'], SIN_PA['sarcopenia'], alternative='two-sided')
print("P-valor entre CON_PA_MECVV y SIN_PA", p_value_23)

P-valor entre CON_PA_MECVV y SIN_PA 0.45734646700671056


## Trastorno del sueño

In [199]:
ts_CON_PA= (CON_PA['problsueño'] == 1.0).mean() * 100
print(f"Porcentaje de trastornos del sueño: {ts_CON_PA:.2f}%")

Porcentaje de trastornos del sueño: 0.00%


In [200]:
ts_CON_PA_MECVV= (CON_PA_MECVV['problsueño'] == 1.0).mean() * 100
print(f"Porcentaje de trastornos del sueño: {ts_CON_PA_MECVV:.2f}%")

Porcentaje de trastornos del sueño: 0.00%


In [201]:
ts_SIN_PA= (SIN_PA['problsueño'] == 1.0).mean() * 100
print(f"Porcentaje de trastornos del sueño: {ts_SIN_PA:.2f}%")

Porcentaje de trastornos del sueño: 0.00%


## Dolor cronico

In [202]:
dc_CON_PA= (CON_PA['dolor_cron'] == 1.0).mean() * 100
print(f"Porcentaje de dolor cronico: {dc_CON_PA:.2f}%")

Porcentaje de dolor cronico: 0.00%


In [203]:
dc_CON_PA_MECVV= (CON_PA_MECVV['dolor_cron'] == 1.0).mean() * 100
print(f"Porcentaje de dolor cronico: {dc_CON_PA_MECVV:.2f}%")

Porcentaje de dolor cronico: 0.00%


In [204]:
dc_SIN_PA= (SIN_PA['dolor_cron'] == 1.0).mean() * 100
print(f"Porcentaje de dolor cronico: {dc_SIN_PA:.2f}%")

Porcentaje de dolor cronico: 0.00%


## Iatrogenico

In [205]:
iatr_CON_PA= (CON_PA['iatrogenico'] == 1.0).mean() * 100
print(f"Porcentaje de iatrogenico: {iatr_CON_PA:.2f}%")

Porcentaje de iatrogenico: 0.00%


In [206]:
iatr_CON_PA_MECVV= (CON_PA_MECVV['iatrogenico'] == 1.0).mean() * 100
print(f"Porcentaje de iatrogenico: {iatr_CON_PA_MECVV:.2f}%")

Porcentaje de iatrogenico: 0.00%


In [207]:
iatr_SIN_PA= (SIN_PA['iatrogenico'] == 1.0).mean() * 100
print(f"Porcentaje de iatrogenico: {iatr_SIN_PA:.2f}%")

Porcentaje de iatrogenico: 0.00%


## Estreñimiento

In [208]:
est_CON_PA= (CON_PA['estreñimiento'] == 1.0).mean() * 100
print(f"Porcentaje de estreñimiento: {est_CON_PA:.2f}%")

Porcentaje de estreñimiento: 0.00%


In [209]:
est_CON_PA_MECVV= (CON_PA_MECVV['estreñimiento'] == 1.0).mean() * 100
print(f"Porcentaje de estreñimiento: {est_CON_PA_MECVV:.2f}%")

Porcentaje de estreñimiento: 0.00%


In [210]:
est_SIN_PA= (SIN_PA['estreñimiento'] == 1.0).mean() * 100
print(f"Porcentaje de estreñimiento: {est_SIN_PA:.2f}%")

Porcentaje de estreñimiento: 0.00%


# Historial medico/comorbilidades